Aim of this notebook is to check saving and loading Frames object 

In [2]:
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple
from tracklet_claude import *

def create_dummy_bbox(frame_id: int, x_base: float = 0.0, y_base: float = 0.0) -> BoundingBox:
    """Create a dummy bounding box with some random variations"""
    return BoundingBox(
        x=x_base + np.random.uniform(-2, 2),
        y=y_base + np.random.uniform(-2, 2),
        l=np.random.uniform(3, 5),
        w=np.random.uniform(1.5, 2.5),
        theta=np.random.uniform(-np.pi, np.pi),
        frame_id=frame_id
    )

def create_dummy_tracklet(id: int, num_frames: int, start_frame: int) -> Tracklet:
    """Create a dummy tracklet with consistent motion"""
    x_base = np.random.uniform(-20, 20)
    y_base = np.random.uniform(-20, 20)
    x_vel = np.random.uniform(-1, 1)
    y_vel = np.random.uniform(-1, 1)
    
    boxes = []
    velocities = []
    
    for i in range(num_frames):
        x = x_base + x_vel * i
        y = y_base + y_vel * i
        box = create_dummy_bbox(start_frame + i, x, y)
        boxes.append(box)
        if i > 0:
            velocities.append((x_vel + np.random.uniform(-0.1, 0.1), 
                             y_vel + np.random.uniform(-0.1, 0.1)))
    
    return Tracklet(
        id=id,
        boxes=boxes,
        confidence=np.random.uniform(0.6, 1.0),
        velocities=velocities,
        length=num_frames
    )

def create_dummy_frame(frame_id: int, num_detections: int, num_tracklets: int) -> Frame:
    """Create a dummy frame with random detections and tracklets"""
    detections = [create_dummy_bbox(frame_id) for _ in range(num_detections)]
    
    # Create tracklets with varying lengths
    tracklets = [
        create_dummy_tracklet(
            id=i,
            num_frames=np.random.randint(2, 6),
            start_frame=max(0, frame_id - np.random.randint(0, 3))
        )
        for i in range(num_tracklets)
    ]
    
    frame = Frame(detections)
    frame.tracklets = tracklets
    return frame

def create_dummy_tracking_data(num_frames: int = 5, 
                             detections_per_frame: int = 3,
                             tracklets_per_frame: int = 2) -> Tuple[List[Frame], List[Frame]]:
    """
    Create dummy forward and backward tracking results for testing
    
    Args:
        num_frames: Number of frames to generate
        detections_per_frame: Number of detections per frame
        tracklets_per_frame: Number of tracklets per frame
        
    Returns:
        Tuple of (forward_results, backward_results)
    """
    forward_results = [
        create_dummy_frame(i, detections_per_frame, tracklets_per_frame)
        for i in range(num_frames)
    ]
    
    backward_results = [
        create_dummy_frame(i, detections_per_frame, tracklets_per_frame)
        for i in range(num_frames)
    ]
    
    return forward_results, backward_results

In [8]:
import json
from dataclasses import asdict
import numpy as np
import pickle
import os

def serialize_bbox(bbox):
    """Convert BoundingBox object to dictionary"""
    return {
        'x': float(bbox.x),
        'y': float(bbox.y),
        'l': float(bbox.l),
        'w': float(bbox.w),
        'theta': float(bbox.theta),
        'id': int(bbox.id),
        'temporal_score': int(bbox.temporal_score),
        'frame_id': int(bbox.frame_id)
    }

def serialize_tracklet(tracklet):
    """Convert Tracklet object to dictionary"""
    return {
        'id': int(tracklet.id),
        'boxes': [serialize_bbox(box) for box in tracklet.boxes],
        'confidence': float(tracklet.confidence),
        'velocities': [[float(vx), float(vy)] for vx, vy in tracklet.velocities],
        'length': int(tracklet.length)
    }

def serialize_frame(frame):
    """Convert Frame object to dictionary"""
    return {
        'detections': [serialize_bbox(det) for det in frame.detections],
        'tracklets': [serialize_tracklet(track) for track in frame.tracklets]
    }

def save_tracking_results(forward_results, backward_results, save_path_prefix, json=False):
    """
    Save forward and backward tracking results to files.
    
    Args:
        forward_results: List of Frame objects from forward tracking
        backward_results: List of Frame objects from backward tracking
        save_path_prefix: String prefix for save paths (without extension)
    """
    forward_filepath = os.path.join(save_path_prefix, 'forward_test.pkl')
    backward_filepath = os.path.join(save_path_prefix, 'backward_test.pkl')
    if json:
        # Save as JSON for human readability
        forward_data = [serialize_frame(frame) for frame in forward_results]
        backward_data = [serialize_frame(frame) for frame in backward_results]
        
        with open(forward_filepath, 'w') as f:
            json.dump(forward_data, f, indent=2)
        
        with open(backward_filepath, 'w') as f:
            json.dump(backward_data, f, indent=2)
        
    # Also save as pickle for easier loading in Python
    with open(forward_filepath, 'wb') as f:
        pickle.dump(forward_results, f)
        
    with open(backward_filepath, 'wb') as f:
        pickle.dump(backward_results, f)

In [12]:
# Generate dummy data
forward_results, backward_results = create_dummy_tracking_data()

# Save the dummy data using the previously defined save function
save_tracking_results(forward_results, backward_results, os.path.join(os.getcwd(), "saved_tracklets"))



In [13]:
load_path = os.path.join(os.getcwd(), "saved_tracklets")
# Optional: Load and verify the pickle files
with open(os.path.join(load_path,"forward_test.pkl"), 'rb') as f:
    loaded_forward = pickle.load(f)
print(f"\nLoaded {len(loaded_forward)} frames from forward tracking data")
print(f"First frame has {len(loaded_forward[0].detections)} detections and {len(loaded_forward[0].tracklets)} tracklets")


Loaded 5 frames from forward tracking data
First frame has 3 detections and 2 tracklets


In [19]:
type(loaded_forward[0].detections[0])

tracklet_claude.BoundingBox

In [7]:
import numpy as np
forward_dimensions_lst = [(1.2, 3.4), (5.6, 7.8), (9.0, 1.1)]
result = np.percentile(forward_dimensions_lst, 95, axis=0)
print(result)
# Output:
# [8.6, 7.2]  # Example output: 95th percentile for each column


[8.66 7.36]


In [ ]:
from enum import Enum
class Corner(Enum):
    REAR_LEFT = 0
    REAR_RIGHT = 1
    FRONT_RIGHT = 2
    FRONT_LEFT = 3


In [15]:
corner_value = 0

corner = Corner(corner_value)
print(corner)

Corner.REAR_LEFT


In [16]:
import numpy as np

# Example: List of numpy arrays of shape (4, 2)
list_of_arrays = [
    np.array([[1, 2], [3, 4], [5, 6], [7, 8]]),
    np.array([[9, 10], [11, 12], [13, 14], [15, 16]]),
    np.array([[17, 18], [19, 20], [21, 22], [23, 24]])
]

# Convert the list to a numpy array of shape (N, 4, 2)
array = np.stack(list_of_arrays)

print("Resulting Array Shape:", array.shape)
print("Resulting Array:")
print(array)


Resulting Array Shape: (3, 4, 2)
Resulting Array:
[[[ 1  2]
  [ 3  4]
  [ 5  6]
  [ 7  8]]

 [[ 9 10]
  [11 12]
  [13 14]
  [15 16]]

 [[17 18]
  [19 20]
  [21 22]
  [23 24]]]
